In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import os
os.chdir(r"/content/drive/MyDrive/WorkShopsSber")

In [ ]:
!unzip -d . dataset.txt.zip

Archive:  dataset.txt.zip
  inflating: ./dataset.txt           


In [ ]:
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 18.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=0c3eb4a64c6a4d60d4547d97b2e4304ec57a6cc3f1606bf742c9b36fe22fd05c
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
# https://www.kaggle.com/datasets/alexandersemiletov/toxic-russian-comments?resource=download
# https://habr.com/ru/articles/542624/

In [ ]:
import re
import pandas as pd
import pymorphy2
import nltk
import numpy as np
import lightgbm as lgb
import hyperopt
import copy

from hyperopt import hp, fmin, tpe, Trials, STATUS_OK, STATUS_FAIL
from sklearn.pipeline import Pipeline
from tqdm import tqdm
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score
from functools import partial
from itertools import chain
from collections import Counter, OrderedDict

In [ ]:
nltk.download('stopwords')
russian_stopwords = set(stopwords.words("russian"))
russian_stopwords = russian_stopwords - set(["не"])
"не" in russian_stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


False

### work with data

In [ ]:
data = open(r"dataset.txt", 'r').read().split('\n')
reg = r"__label__[A-Z]+"
data = [[re.compile(reg).sub("", i).strip(), re.compile(reg).findall(i)]
        for i in tqdm(data)]
data = pd.DataFrame(data, columns=['comment', 'target'])
data_exploded = data.explode(['target']).reset_index(drop=True)

100%|██████████| 248291/248291 [00:01<00:00, 145072.25it/s]


In [ ]:
dataset = data_exploded[data_exploded['target'].isin(['__label__NORMAL', '__label__INSULT'])]
print("dataset shape: ", dataset.shape)
data_exploded['target'].value_counts()
# сначала работаем только с INSULT

dataset shape:  (240511, 2)


target
__label__NORMAL       203685
__label__INSULT        36826
__label__THREAT        12027
__label__OBSCENITY      4261
Name: count, dtype: int64

In [ ]:
dataset.head()

,comment,target
0,скотина! что сказать,__label__INSULT
1,я сегодня проезжала по рабочей и между домами ...,__label__NORMAL
2,очередной лохотрон. зачем придумывать очередно...,__label__NORMAL
3,"ретро дежавю ... сложно понять чужое сердце , ...",__label__NORMAL
4,а когда мы статус агрогородка получили?,__label__NORMAL


In [ ]:
mrp = pymorphy2.MorphAnalyzer(lang='ru')

def text_preprocessor(sent, reg=r"(?:[a-zA-Zа-яА-Я0-9-]+|[!])", lemmatizer=mrp, stpwrd=russian_stopwords):
    sent = re.compile(reg).findall(str(sent))
    func_lemma = lambda x: lemmatizer.parse(x)[0].normal_form
    sent = list(map(func_lemma, sent))
    sent = list(filter(lambda x: x not in russian_stopwords, sent))
    sent = " ".join(sent)
    sent = re.compile(r"не ").sub("не_", sent)
    return sent

text_preprocessor("скотина! что сказать")
text_preprocessor("я сегодня проезжала по рабочей и между домами")
text_preprocessor("я сегодня не иду на работу")

'сегодня не_идти работа'

In [ ]:
# dataset['comment_postprocessed'] = dataset['comment'].apply(lambda x: text_preprocessor(x))
dataset['comment_postprocessed'] = [text_preprocessor(i) for i in tqdm(dataset['comment'].values)]

100%|██████████| 240511/240511 [16:00<00:00, 250.49it/s]
<ipython-input-65-feb07b90183d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['comment_postprocessed'] = [text_preprocessor(i) for i in tqdm(dataset['comment'].values)]


In [ ]:
# dataset.to_csv(r"postprocessed_dataset.csv", index=False)
# dataset = pd.read_csv(r"postprocessed_dataset.csv")
dataset['target_binary'] = dataset['target'].apply(lambda x: 1 if x == '__label__INSULT' else 0)

<ipython-input-75-a9b5ea2e8037>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['target_binary'] = dataset['target'].apply(lambda x: 1 if x == '__label__INSULT' else 0)


In [ ]:
dataset.head()
#dataset[dataset.target_binary == 1]

,comment,target,comment_postprocessed,target_binary
0,скотина! что сказать,__label__INSULT,скотина ! сказать,1
1,я сегодня проезжала по рабочей и между домами ...,__label__NORMAL,сегодня проезжать рабочий дом снитенко гомолыс...,0
2,очередной лохотрон. зачем придумывать очередно...,__label__NORMAL,очередной лохотрон придумывать очередной налог...,0
3,"ретро дежавю ... сложно понять чужое сердце , ...",__label__NORMAL,ретро дежавю сложно понять чужое сердце лиш ощ...,0
4,а когда мы статус агрогородка получили?,__label__NORMAL,статус агрогородок получить,0


In [ ]:
X = dataset['comment_postprocessed'].values
y = dataset['target_binary'].values
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=123,
                                                    test_size=0.15,
                                                    stratify=y)

### pipeline formation

In [ ]:
rstate = np.random.default_rng(42)

In [ ]:
# итоговая модель
model = Pipeline([('embedder', TfidfVectorizer()),
                  ('classifier', lgb.LGBMClassifier(max_depth=3, n_estimators=10, class_weight='balanced'))])

In [ ]:
boosting_type = ['gbdt', 'dart', 'rf']
max_depth = [2, 3, 4, 5, 6, 7, 10, -1]
n_estimators = np.arange(5, 125, 5)
search_space = {
                'embedder__min_df': hp.uniform(label='min_df', low=0.001, high=0.05),
                'embedder__max_df': hp.uniform(label='max_df', low=0.33, high=0.6),
                'classifier__boosting_type': hp.choice(label='boosting_type', options=boosting_type),
                'classifier__max_depth': hp.choice(label='max_depth', options=max_depth),
                'classifier__n_estimators': hp.choice(label='n_estimators', options=n_estimators),
                }

In [ ]:
def objective(params, pipeline,  X_train, y_train):
    """
    Кросс-валидация с текущими гиперпараметрами

    :params: гиперпараметры
    :pipeline: модель
    :X_train: матрица признаков
    :y_train: вектор меток объектов
    :return: средняя точность на кросс-валидации
    """
    # задаём модели требуемые параметры
    pipeline.set_params(**params)
    # задаём параметры кросс-валидации (стратифицированная 4-фолдовая с перемешиванием)
    skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=123)
    # проводим кросс-валидацию
    try:
        score = cross_val_score(estimator=pipeline, X=X_train, y=y_train,
                                scoring='roc_auc', cv=skf, n_jobs=4)
    except:
        return {'loss': 0, 'params': params, 'status': STATUS_FAIL}
    # возвращаем результаты, которые записываются в Trials()
    return {'loss': -score.mean(), 'params': params, 'status': STATUS_OK}

In [ ]:
# запускаем hyperopt
trials = Trials()
best = fmin(
          # функция для оптимизации
            fn=partial(objective, pipeline=model, X_train=X_train[:10000], y_train=y_train[:10000]),
          # пространство поиска гиперпараметров
            space=search_space,
          # алгоритм поиска
            algo=tpe.suggest,
          # число итераций
          # (можно ещё указать и время поиска)
            max_evals=40,
          # куда сохранять историю поиска
            trials=trials,
          # random state
            rstate=rstate,
          # progressbar
            show_progressbar=True
        )

100%|██████████| 40/40 [00:42<00:00,  1.06s/trial, best loss: -0.7864031925162047]


In [ ]:
statuses = [i['result']['status'] for i in trials.trials]
# интересно почему упало?
outputs_hp = [i for i, j in zip(trials.trials, statuses) if j == 'fail']
print("Упавшие варианты: ", len(outputs_hp))

best_reset = dict()
best_reset['classifier__boosting_type'] = boosting_type[best['boosting_type']]
best_reset['classifier__max_depth'] = max_depth[best['max_depth']]
best_reset['classifier__n_estimators'] = n_estimators[best['n_estimators']]
best_reset['embedder__max_df'] = best['max_df']
best_reset['embedder__min_df'] = best['min_df']

Упавшие варианты:  8


In [ ]:
best_reset

{'classifier__boosting_type': 'gbdt',
 'classifier__max_depth': 3,
 'classifier__n_estimators': 75,
 'embedder__max_df': 0.4928643309276279,
 'embedder__min_df': 0.0012850043662593}

In [ ]:
model.set_params(**best_reset)
model.fit(X_train[:10000], y_train[:10000])

In [ ]:
print("roc-auc train: ", roc_auc_score(y_train[:10000], model.predict_proba(X_train[:10000])[:, 1]))
print("roc-auc test: ", roc_auc_score(y_test, model.predict_proba(X_test)[:, 1]))

roc-auc train:  0.8252256262884073
roc-auc test:  0.8120062487775127


### NN architectures word2vec by words(not morphems)

In [ ]:
import tensorflow as tf
from tensorflow.keras import models as md
from tensorflow.keras import layers as ls
from tensorflow.keras import optimizers as opt
from tensorflow.keras import metrics as mt
from tensorflow.keras import callbacks as clb
from tensorflow.keras import losses as lss

import matplotlib.pyplot as plt

In [ ]:
class Reconfigure:
    def __init__(self, max_shape=18):
        self.vocab = None
        self.length_hist = None
        self.max_shape = max_shape

    def __text2nums(self, ln_vocab, data):
        output = []
        for text in data:
            prom = []
            for word in text.split():
                if word in self.vocab:
                    prom.append(self.vocab[word])
                else:
                    prom.append(ln_vocab)
            if len(prom) < self.max_shape:
                prom = prom + [ln_vocab + 1]*(self.max_shape - len(prom))
            output.append(prom[:self.max_shape])
        return np.array(output, dtype=int)

    def fit_transform(self, data):
        self.vocab = dict(enumerate(set(chain(*[i.split() for i in data]))))
        ln_vocab = len(self.vocab)
        self.vocab[ln_vocab] = 'UNKNOWN'
        self.vocab[ln_vocab + 1] = 'ZERO_PAD'
        self.vocab = {self.vocab[i]: i for i in self.vocab}
        self.length_hist = sorted(dict(Counter([len(i.split()) for i in data])).items(), key=lambda x: -x[1])
        return self.__text2nums(ln_vocab, data)

    def transform(self, data):
        assert not self.vocab is None, "Сначала нужно обучить реконфигуратор!"
        ln_vocab = len(self.vocab)
        return self.__text2nums(ln_vocab, data)

In [ ]:
e = Reconfigure()
X_transformed = e.fit_transform(X_train[:10000])

In [ ]:
def model_nn(shape_=X_transformed.shape, vocab_size=len(e.vocab), n_units=e.max_shape):
    input = ls.Input(shape_[1])
    x = ls.Embedding(vocab_size, 300)(input)
    x = ls.LSTM(n_units, activation="tanh", recurrent_activation="sigmoid", dropout=0.0, return_sequences=False)(x)
    # x = ls.Dense(128)(x)
    x = ls.Dense(1, activation='sigmoid')(x)
    model = md.Model(inputs=[input], outputs=[x])
    return model

md1 = model_nn()
md1.summary()

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_17 (InputLayer)       [(None, 18)]              0         
                                                                 
 embedding_10 (Embedding)    (None, 18, 300)           6182400   
                                                                 
 lstm_6 (LSTM)               (None, 18)                22968     
                                                                 
 dense_10 (Dense)            (None, 1)                 19        
                                                                 
Total params: 6205387 (23.67 MB)
Trainable params: 6205387 (23.67 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
md1.compile(
    optimizer="rmsprop",
    loss=lss.BinaryCrossentropy(),
    loss_weights=None,
    metrics=['AUC'],
)
md1.fit(X_transformed, y_train[:100000], batch_size=32, epochs=10, validation_split=0.15)

Epoch 1/10
266/266 [==============================] - 15s 47ms/step - loss: 0.3917 - auc: 0.6648 - val_loss: 0.2566 - val_auc: 0.9083
Epoch 2/10
266/266 [==============================] - 11s 43ms/step - loss: 0.1703 - auc: 0.9538 - val_loss: 0.1775 - val_auc: 0.9359
Epoch 3/10
266/266 [==============================] - 11s 42ms/step - loss: 0.0705 - auc: 0.9901 - val_loss: 0.2589 - val_auc: 0.9363
Epoch 4/10
266/266 [==============================] - 12s 44ms/step - loss: 0.0315 - auc: 0.9960 - val_loss: 0.2496 - val_auc: 0.9220
Epoch 5/10
266/266 [==============================] - 11s 43ms/step - loss: 0.0159 - auc: 0.9982 - val_loss: 0.3788 - val_auc: 0.9125
Epoch 6/10
266/266 [==============================] - 12s 44ms/step - loss: 0.0104 - auc: 0.9984 - val_loss: 0.5199 - val_auc: 0.9144
Epoch 7/10
266/266 [==============================] - 15s 56ms/step - loss: 0.0053 - auc: 0.9994 - val_loss: 0.4458 - val_auc: 0.9025
Epoch 8/10
266/266 [==============================] - 11s 40ms

### berts